In [ ]:
%pip install transformers sentencepiece accelerate bitsandbytes einops
%pip install transformers==4.19.0
%pip install transformers --upgrade


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code = True)

In [ ]:

model = transformers.AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    torch_dtype = "auto",
    device_map = "auto",
    trust_remote_code = True
)
prompt = """Sally (a girl) has 3 brothers. Each brother has 2 sisters. How many sisters does Sally have? solution"""

In [ ]:
with torch.no_grad():
    # Ensure token_ids has the correct data type (Long) for the embedding layer
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(dtype=torch.long)
   
    # Convert the model to full precision
    model = model.to(dtype=torch.float32)
   
    # Move token_ids to the appropriate device
    token_ids = token_ids.to(model.device)
   
    output_ids = model.generate(
        token_ids,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.3
    )

In [ ]:
output = tokenizer.decode(output_ids[0][token_ids.size(1):])
split_text = output.split("<|endoftext|>")
print(split_text[0].strip())